# Recommendation Systems Exhibition

In [1]:
# Import packages used throughout
from mxnet import autograd, gluon, np, npx
from mxnet.gluon import nn
import mxnet as mx
import pandas as pd

from data.import_data import split_and_load_ml100k
from methods.utils import Accumulator, try_all_gpus

from methods.matrix_factorization import MF, evaluator, train_recsys_rating

npx.set_np()

## Part 1: MovieLens Dataset

In [2]:
# Import data
names = ["user_id", "item_id", "rating", "timestamp"]
data = pd.read_csv("data/u.data", delimiter='\t', names = names, engine = "python")
num_users = data.user_id.unique().shape[0]
num_items = data.item_id.unique().shape[0]

## Part 2: Matrix Factorization

In [4]:
devices = try_all_gpus()
num_users, num_items, train_iter, test_iter = split_and_load_ml100k(
    data, num_users, num_items, test_ratio = 0.1, batch_size = 512)
net = MF(30, num_users, num_items)
net.initialize(ctx=devices, force_reinit=True, init=mx.init.Normal(0.01))
lr, num_epochs, wd, optimizer = 0.002, 20, 1e-5, 'adam'
loss = gluon.loss.L2Loss()
trainer = gluon.Trainer(net.collect_params(), optimizer,
                        {"learning_rate": lr, 'wd': wd})
train_recsys_rating(net, train_iter, test_iter, loss, trainer, num_epochs,
                    devices, evaluator)

NameError: name 'evaluator' is not defined